# Volatility Measurement and Forecasting

The purpose of this tutorial is to introduce various estimators for forecasting volatility.  This material is closely related to a the following white papers:

1. *Volatility Modeling and Trading* - Artur Sepp, 2016

2. *Measuring Historic Volatility* - Colin Bennet & Miguel Gil, 2012

Our main objective will be to implement the code for various historical volatility estimators.  To test our work, we will attempt to replicate some of Sepp's results for *weekly* volatility forecasts for SPY (see pp 38-43).

## Loading Packages

Let's begin by loading the packages that we will need.

In [1]:
##> import pandas as pd
##> import pandas_datareader as pdr
##> import numpy as np
##> import sklearn
##> pd.options.display.max_rows = 10




## Reading-In SPY Data From Yahoo Finance

Sepp's analysis covers the following dates: 1/1/2005 - 4/2/2016.  So, let's grab these SPY prices from Yahoo Finance using `pandas_datareader`.

In [2]:
##> df_spy = pdr.get_data_yahoo('SPY', start = '2004-12-31', end = '2016-04-02').reset_index()
##> df_spy.columns = df_spy.columns.str.lower().str.replace(' ', '_')
##> df_spy.rename(columns = {'date':'trade_date'}, inplace = True)
##> df_spy.insert(0, 'ticker', 'SPY')
##> df_spy




## Calculating Daily Returns & Realized Volatility

The volatility estimators that we will implement will involve various daily returns.  Let's calculate in the following block of code.

In [3]:
##> df_spy['dly_ret'] = np.log(df_spy['close']).diff()
##> df_spy['overnight'] = np.log(df_spy['open']) - np.log(df_spy['close']).shift(1)
##> df_spy['open_close'] = np.log(df_spy['close']) - np.log(df_spy['open'])
##> df_spy = df_spy[1:].reset_index(drop = True)
##> df_spy




## Organizing Dates for Backtest

Organizing dates is an important step in a historical analysis.  

We are performing a weekly analysis, which means that in later steps we will performing aggregation calculations of daily calculations grouped into weeks.  Thus, it will be useful to add a column to our data that will allow us to group by weeks.

The key to our approach will be to use the `.dt.weekday` attribute of the `trade_date` columns.  In the following code, `ser_weekday` is a series that contains the weekday associated with each date.  Notice that Monday is encoded by `0` and Friday is encoded by `4`.

In [4]:
##> ser_weekday = df_spy['trade_date'].dt.weekday
##> ser_weekday




The following code is a simple for-loop that has the effect of creating a week-number for each week.

In [5]:
##> lst_week_num = []
##> ix_week = 0
##> lst_week_num.append(ix_week)
##> for ix in range(0, len(ser_weekday) - 1):
##>     prev_day = ser_weekday[ix]
##>     curr_day = ser_weekday[ix + 1]
##>     if curr_day < prev_day:
##>         ix_week = ix_week + 1
##>     lst_week_num.append(ix_week)
##> np.array(lst_week_num) # I use the array function simply because it looks better when it prints




Let's now insert the week numbers into `df_spy`.

In [6]:
##> df_spy.insert(2, 'week_num', lst_week_num)
##> df_spy




**Discussion Question:** The `pandas.Series.dt.week` attribute gives the *week-of-the-year* for a give trade-date.  My initial idea was to use `.dt.week` and `dt.year` for my grouping, but I ran into an issue.  Do you have any guesses as to what the issue was?

We can now use `groupby()` to calculate the starting and ending dates for each week.

In [7]:
##> df_start_end = \
##>     (
##>     df_spy.groupby(['week_num'], as_index = False)[['trade_date']].agg([min, max])['trade_date']
##>     .rename(columns = {'min':'week_start', 'max':'week_end'})
##>     .reset_index()
##>     .rename(columns = {'index':'week_num'})
##>     )
##> df_start_end




Let's merge this data into `df_spy`.

In [8]:
##> df_spy = df_spy.merge(df_start_end)
##> df_spy




## Calculating Weekly Realized Volatility

Now that we have a `week_num` associated with each `trade_date`, we can use `groupby()` to calculate the realized volatility.

These weekly realized volatilities are the labels that we will be predicting later in our analysis.

In [9]:
##> df_realized = \
##>     (
##>     df_spy
##>         .groupby(['week_num', 'week_start', 'week_end'], as_index = False)[['dly_ret']].agg(lambda x: np.std(x) * np.sqrt(252))
##>         .rename(columns = {'dly_ret':'realized_vol'})
##>     )
##> df_realized = df_realized[1:]
##> df_realized




## Close-to-Close Estimator

The first estimator that we will implement is the simlple close-to-close.

In [10]:
##> def close_to_close(r):
##>     T = r.shape[0]
##>     r_bar = r.mean()
##>     vol = np.sqrt((1 / (T - 1)) * ((r - r_bar) ** 2).sum()) * np.sqrt(252)
##>     return(vol)




Notice that `close_to_close()` is an aggregation function that takes in an array of daily returns, and gives back a number.  Thus, in order to calculate weekly estimates we use `close_to_close()` as the aggregation function applied to a `.groupby()`.

In [11]:
##> df_close_to_close = \
##>     (
##>     df_spy
##>         .groupby(['week_num', 'week_start', 'week_end'], as_index = False)[['dly_ret']]
##>         .agg(close_to_close)
##>         .rename(columns = {'dly_ret':'close_to_close'})
##>     )
##> df_close_to_close = df_close_to_close[0:-1]
##> df_close_to_close




**Discussion Question:** What is the difference between our custom defined `close_to_close()` and `np.std()`?  Why does it make sense to use `np.std()` to calculate realized volatility?

In Sepp 2016, the author uses the $R^2$ between the forecasts and the realized labels as a means of assessing the quality of a particular estimator.  Let's utilize `sklearn` to do the same.

We being by importing the `LinearRegression` constructor and instantiating a model.

In [12]:
##> from sklearn.linear_model import LinearRegression
##> mdl_reg = LinearRegression(fit_intercept = True)




Next, let's organize our features and labels.

In [13]:
##> X = df_close_to_close[['close_to_close']]
##> y = df_realized['realized_vol']




We can now fit the model.

In [14]:
##> mdl_reg.fit(X, y)



The `.score()` method of a `LinearRegression` model returns the $R^2$.

In [15]:
##> mdl_reg.score(X, y)




And we can examine the slope and intercept of our model as follows:

In [16]:
##> print(mdl_reg.intercept_)
##> print(mdl_reg.coef_)




**Discussion Question:** Compare our results with Sepp's.  How do you feel about the amount of discrpency in our results?

## Parkinson

The next estimator that we implement is the Parkinson.

In [17]:
##> def parkinson(hl):
##>     T = hl.shape[0]
##>     high = hl.high
##>     low = hl.low
##>     vol = np.sqrt(np.sum((np.log(high / low) ** 2)) * (1 / (4 * np.log(2))) / T) * np.sqrt(252)
##>     return(vol)




Let's apply our function to a single weeks worth of data in `df_spy`.

In [18]:
##> parkinson(df_spy.query('week_num == 0')[['high', 'low']])



From a programming standpoint, the Parkinson estimate is a little bit different because it is an aggregation function that takes in two columns (`high` and `low`) and returns a single number.  

For this reason, we will need to use `.apply()` rather than `.agg()`.

In [19]:
##> df_parkinson = \
##>     (
##>     df_spy.groupby(['week_num', 'week_start', 'week_end'])[['high', 'low']].apply(parkinson)
##>     .to_frame().reset_index()
##>     .rename(columns = {0:'parkinson'})
##>     )
##> df_parkinson = df_parkinson[:-1]
##> df_parkinson




Next, let's fit a linear regression to the parkinson forecasts and the realized volatilities.

In [ ]:
##> from sklearn.linear_model import LinearRegression
##> mdl_reg = LinearRegression(fit_intercept = True)
##> X = df_parkinson[['parkinson']]
##> y = df_realized['realized_vol']
##> mdl_reg.fit(X, y)




**Code Challenge:** Check the $R^2$ and coefficients and compare them with Sepp's.  What are your thoughts on the discrepancies?

## Garman-Klass

The next estimator is the Garman-Klass.

In [20]:
##> def garman_klass(ohlc):
##>     T = ohlc.shape[0]
##>     o = ohlc.open
##>     h = ohlc.high
##>     l = ohlc.low
##>     c = ohlc.close
##>     vol = np.sqrt(np.sum((0.5 * np.log(h / l) ** 2) - ((2 * np.log(2) - 1) * np.log(c / o) ** 2)) / T) * np.sqrt(252)
##>     return(vol)




Let's check that the function works for a single week of data.

In [21]:
##> garman_klass(df_spy.query('week_num == 0')[['open', 'high', 'low', 'close']])




The Garman-Klass estimator takes in four different columns to produce a single numeric estimate, thus we have to use `.apply()`.

In [22]:
##> df_garman_klass = \
##>     (
##>     df_spy.groupby(['week_num', 'week_start', 'week_end'])[['open', 'high', 'low', 'close']].apply(garman_klass)
##>     .to_frame().reset_index()
##>     .rename(columns = {0:'garman_klass'} )
##>     )
##> df_garman_klass = df_garman_klass[:-1]
##> df_garman_klass




Next let's check for the goodness of predictions by fitting a linear regression and calculating the $R^2$.

In [23]:
##> from sklearn.linear_model import LinearRegression
##> mdl_reg = LinearRegression(fit_intercept = True)
##> X = df_garman_klass[['garman_klass']]
##> y = df_realized['realized_vol']
##> mdl_reg.fit(X, y)
##> mdl_reg.score(X, y)

## Rogers-Satchell

**Code Challenge:** Implement the Rogers-Satchell model, and calculate the $R^2$ between the forecasts and the realized.

## Yang-Zhang

And finally, let's repeat thes same steps for the Yang-Zang estimator.

In [24]:
##> def yang_zhang(ohlc_on_oc):
##>     T = ohlc_on_oc.shape[0]
##>     ohlc = ohlc_on_oc[['open', 'high', 'low', 'close']]
##>     on = ohlc_on_oc.overnight
##>     oc = ohlc_on_oc.open_close
##>     
##>     var_overnight = (close_to_close(on) / np.sqrt(252)) ** 2
##>     var_open_close = (close_to_close(oc) / np.sqrt(252)) ** 2
##>     var_rogers_satchell = (rogers_satchell(ohlc) / np.sqrt(252)) ** 2
##>     
##>     c = 0.34 / (1.34 + (T + 1)/(T - 1))
##>     
##>     vol = np.sqrt((var_overnight) + (c * var_open_close) + ((1 - c) * (var_rogers_satchell))) * np.sqrt(252)
##>     
##>     return(vol)





Checking the function on a single week of data.

In [26]:
##> yang_zhang(df_spy.query('week_num == 0')[['open', 'high', 'low', 'close', 'overnight', 'open_close']])



Calculating weekly forecasts using `.groupby()` and `.apply()`.

In [27]:
##> df_yang_zhang = \
##>     (
##>     df_spy.groupby(['week_num', 'week_start', 'week_end'])[['open', 'high', 'low', 'close', 'overnight', 'open_close']].apply(yang_zhang)
##>     .to_frame().reset_index()
##>     .rename(columns = {0:'yang_zhang'} )
##>     )
##> df_yang_zhang = df_yang_zhang[:-1]
##> df_yang_zhang




Let's check the performance of Yang-Zang by checking the $R^2$ of the fitted regression.

In [28]:
##> from sklearn.linear_model import LinearRegression
##> mdl_reg = LinearRegression(fit_intercept = True)
##> X = df_yang_zhang[['yang_zhang']]
##> y = df_realized['realized_vol']
##> mdl_reg.fit(X, y)
##> mdl_reg.score(X, y)




**Code Challenge:** There is a short-hand identity for $R^2$ that would allow us to not have to bother with `sklearn`.